# 네이버 뉴스 데이터 가져오기

> 1. 네이버 오픈 API로 기사 URL 크롤링
> 2. bs4 + selenium으로 기사 제목 및 내용 크롤링

# 크롬 브라우저가 뜨면 없애지 말고 있는 상태에서 코드 실행하기

In [45]:
import os
import sys
import urllib.request
import requests

news_data = []
page_count = 3

client_id = "GGc4QLn07KQawsbkTYOQ"
client_secret = "JfYfWSHJVd" # 발급받은 client_secret
encText = urllib.parse.quote("파이썬")

for idx in range(page_count):
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&start=" + str(idx *10 + 1) # json 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과


    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()


    if(rescode==200):
    # 기존 네이버 도큐먼트
    #    response_body = response.read()
    #    print(response_body.decode('utf-8'))

        # 기존에 제공하던 도큐먼트에서 수정한 부분 - 링크를 얻고 정보를 받아오기 위해 requests를 씀
        # 우리는 정보를 저장하기 위해 이렇게 수정
        result = requests.get(response.geturl(),
                              headers={"X-Naver-Client-Id":client_id,
                                       "X-Naver-Client-Secret":client_secret}
                             )
        news_data.append(result.json())

    else:
        print("Error Code:" + rescode)

### 네이버 OPEN API를 통해 가져온 데이터 확인하기.

In [46]:
news_data

[{'lastBuildDate': 'Wed, 10 Feb 2021 14:41:38 +0900',
  'total': 9460,
  'start': 1,
  'display': 10,
  'items': [{'title': '더조은컴퓨터학원 일산캠퍼스, 자바프로그래밍 SW개발 국비지원 전액무료취업...',
    'originallink': 'http://www.it-b.co.kr/news/articleView.html?idxno=47331',
    'link': 'http://www.it-b.co.kr/news/articleView.html?idxno=47331',
    'description': '자바/<b>파이썬</b>활용 빅데이터개발과정까지 지속적인 성공적운영과 훈련생들의 높은 만족도를 이끌어 냈으며, 2021년 자바응용SW개발자양성 과정과 과정평가형(정보처리산업기사)를 추가 승인 받아 기존 디자인/영상... ',
    'pubDate': 'Wed, 10 Feb 2021 09:45:00 +0900'},
   {'title': '한국장학진흥원, 빅데이터전문가 및 코딩지도사 자격증 무료수강 이벤트 참여...',
    'originallink': 'http://www.mrepublic.co.kr/news/articleView.html?idxno=63484',
    'link': 'http://www.mrepublic.co.kr/news/articleView.html?idxno=63484',
    'description': '프로그래밍 언어 <b>파이썬</b>과 웹크롤링, 데이터분석 교육 콘텐츠를 찾는 사람들에게 도움되는 다양한 콘텐츠도 만날 수 있다. 사고력을 길러주는 다양한 온라인 코딩지도사자격증 교육 프로그램은 어린이들을 위한... ',
    'pubDate': 'Wed, 10 Feb 2021 00:02:00 +0900'},
   {'title': "'주린이'를 도와라... AI 투자 열풍, 국내 로보어드바이저 앱 5종 소개",
    'ori

In [47]:
print(news_data[0]['items'][0]['link'])

http://www.it-b.co.kr/news/articleView.html?idxno=47331


In [48]:
# 페이지 하나당 리스트로 묶여서 들고옴
# request.Request(url) 는 한 번에 묶어서 들고오는 것임

len(news_data)

3

### 가져온 URL이 네이버 뉴스인지 확인하기.

In [49]:
naver_news_link = []

for page in news_data:
    
    page_news_link = []

    for item in page['items']:
        link = item['link']
        if "naver" in link:
            page_news_link.append(link)
            
    naver_news_link.append(page_news_link)

print(len(naver_news_link))

3


In [50]:
# 한 페이지 당 네이버 뉴스가 1개 / 2개 / 1개 있음

naver_news_link

[['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0012194044'],
 ['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=011&aid=0003869027',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=022&aid=0003550511'],
 ['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=014&aid=0004579584']]

In [51]:
# 페이지 당 몇개의 링크가 있는지 확인

for page in naver_news_link:
    for link in page:
        print(link)

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0012194044
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=011&aid=0003869027
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=022&aid=0003550511
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=014&aid=0004579584


### 네이버 뉴스 기사 크롤링!

In [52]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import notebook
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup 
from urllib.request import urlopen
import urllib
import time

In [53]:
# 가상 크롬드라이버를 불러옴.
# 윈도우 10의 경우 chromedriver.exe
driver = webdriver.Chrome("driver/chromedriver.exe")

- 웹 크롤링 코드
- 이차원 구조로 만들어주면 편함!

In [54]:
naver_news_title = []
naver_news_content = []


for n in notebook.tqdm(range(len(naver_news_link))):
    #print(n)
    news_page_title = []
    news_page_content = []
    
    for idx in notebook.tqdm(range(len(naver_news_link[n]))):
        
        
    ########### 긁어온 URL로 접속하기 ############    
        try:
            driver.get(naver_news_link[n][idx])
            print(naver_news_link[n][idx])
            
        except:
            print("Timeout!")
            continue
        
        
        try:
            response = driver.page_source
            
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제된 경우입니다.")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        ###### 뉴스 타이틀 긁어오기 ######
        
        title = None
        
        try:
            item = soup.find('div', class_="article_info")
            title = item.find('h3', class_="tts_head").get_text()
            #print(title)

        except:
            title = "OUTLINK"
        
        #print(title)
        news_page_title.append(title)
        
        
        ###### 뉴스 본문 긁어오기 ######
        
        doc = None
        text = ""
                
        data = soup.find_all("div", {"class" : "_article_body_contents"})
        if data:
            for item in data:

                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text) # 타입을 string으로 바꾸기 위해 # 안정적인 함수
                doc = ' '.join(text)
   
        else:
            doc = "OUTLINK"
            
        news_page_content.append(doc.replace('\n', ' '))

                
    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)

    time.sleep(2)
    
    
print(naver_news_title[0])
print("==================================")
print(naver_news_content[0])

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0012194044



https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=011&aid=0003869027
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=022&aid=0003550511



https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=014&aid=0004579584


["aSSIST 경영대학원, AI 기반 'AI 전략경영 MBA 과정' 개설"]
['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    ▲ aSSIST 경영대학원(서울과학종합대학원)은 인공지능(AI) 전략경영 경영학석사(MBA) 과정을 개설했다고 9일 밝혔다.     이 과정은 AI의 핵심 원리와 경영 현장에 적용할 수 있는 전략을 교육하며 실제 기업별 AI 활용 및 응용 전략을 끌어내는 것을 목표로 한다. 최신 교과목으로 소프트웨어 비전공 기업 실무자도 쉽게 AI 원리를 익히고 현장에서 AI를 적용하도록 설계됐다.     커리큘럼은 AI, AI 전략, AI 경영학 3가지 분야로 나뉜다.     AI 부문은 AI 분석 도구(오렌지)·AI 분석 도구(파이썬)·AI와 딥러닝·AI와 강화 학습으로 구성됐다.     AI 전략 부문은 AI 기반 전략 프로세스·AI 기반 전략 대안·AI 기반 전략 워크숍·AI 기반 전략 세미나(메커니즘 경영)·AI 기반 전략 세미나(지속 경영·ESG) 등의 과목으로 돼 있다.     AI 경영학 부문은 AI 기반 마케팅·AI 기반 회계학·AI 기반 재무관리·AI 기반 생산관리·AI 기반 인사 조직으로 이뤄져 있다.     주요 교수로 조동성·문휘창 서울대 명예교수, 김문수 aSSIST 경영대학원 부총장, 김성민·김보영·문달주 aSSIST 교수, 홍성인 교수(\'문과생을 위한 파이썬\' 저자) 등이 참여한다.     AI 전략경영 MBA 교육은 주말에만 진행하며 총 24학점을 1년 안에 취득할 수 있도록 설계됐다.     김문수 부총장은 "기업 경영자, 실무자들이 AI 개념과 적용 원리를 손쉽게 학습해 실제 비즈니스 성과를 창출할 수 있도록 AI 경영 

### 크롤링 결과 확인하기.

In [55]:
print(naver_news_title[0])

["aSSIST 경영대학원, AI 기반 'AI 전략경영 MBA 과정' 개설"]


In [56]:
print(naver_news_content[0])

['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    ▲ aSSIST 경영대학원(서울과학종합대학원)은 인공지능(AI) 전략경영 경영학석사(MBA) 과정을 개설했다고 9일 밝혔다.     이 과정은 AI의 핵심 원리와 경영 현장에 적용할 수 있는 전략을 교육하며 실제 기업별 AI 활용 및 응용 전략을 끌어내는 것을 목표로 한다. 최신 교과목으로 소프트웨어 비전공 기업 실무자도 쉽게 AI 원리를 익히고 현장에서 AI를 적용하도록 설계됐다.     커리큘럼은 AI, AI 전략, AI 경영학 3가지 분야로 나뉜다.     AI 부문은 AI 분석 도구(오렌지)·AI 분석 도구(파이썬)·AI와 딥러닝·AI와 강화 학습으로 구성됐다.     AI 전략 부문은 AI 기반 전략 프로세스·AI 기반 전략 대안·AI 기반 전략 워크숍·AI 기반 전략 세미나(메커니즘 경영)·AI 기반 전략 세미나(지속 경영·ESG) 등의 과목으로 돼 있다.     AI 경영학 부문은 AI 기반 마케팅·AI 기반 회계학·AI 기반 재무관리·AI 기반 생산관리·AI 기반 인사 조직으로 이뤄져 있다.     주요 교수로 조동성·문휘창 서울대 명예교수, 김문수 aSSIST 경영대학원 부총장, 김성민·김보영·문달주 aSSIST 교수, 홍성인 교수(\'문과생을 위한 파이썬\' 저자) 등이 참여한다.     AI 전략경영 MBA 교육은 주말에만 진행하며 총 24학점을 1년 안에 취득할 수 있도록 설계됐다.     김문수 부총장은 "기업 경영자, 실무자들이 AI 개념과 적용 원리를 손쉽게 학습해 실제 비즈니스 성과를 창출할 수 있도록 AI 경영 전문가들을 양성하겠다"고 말했다.     이 과정의 봄 학기 원서 접수 기간은 오는 19일까지며 aSSIST 경영대학원 홈페이지에서 할 수 있다. (연합뉴스) ▶네이버에서도 뉴스는 연합뉴스[구독 클릭] ▶[팩트체크]은마아파트 2천300t

In [57]:
print(len(naver_news_title[0]))
print(len(naver_news_content[0]))

1
1


### 크롤링 결과 저장하기.

In [58]:
with open("naver_news_title.pk", "wb") as f:
    pickle.dump(naver_news_title, f)

with open("naver_news_content.pk", "wb") as f:
    pickle.dump(naver_news_content, f)